### Paso 1.4 - Creación de un Pipeline

1. Crear el Pipeline de ingesta de datos **PL_Onprem_ADLS_Ingest**, el cual contendrá una actividad **Copy data**

2. Crearemos un Pipeline más eficiente realizando una **Carga Incremental (Incremental load)** basandonos en la última fecha    
   modificada. Para ello, necesitaremos crear un nuevo dataset de origen llamado **DS_Onprem_File_param** y un dataset de destino llamado **DS_ADLS_ingest_param** que utilizarán parámetros.

#### Crear el Pipeline de ingesta de datos **PL_Onprem_ADLS_Ingest**, el cual contendrá una actividad **Copy data**

En la pestaña **Source** de la actividad **Copy data** tenemos la opción de **Wildcard file path** al cual le ingresamos el comodín ***.csv**, de esta manera tomará TODOS LOS ARCHIVOS CON EXTENSIÓN .CSV en la ruta especificada.

<center><img src="https://i.postimg.cc/X7x9tbLM/adf495.png"></center>

En la pestaña **Sink** no hacemos mayores cambios, más que ingresar el dataset de destino

<center><img src="https://i.postimg.cc/zBmhTXzh/adf496.png"></center>

Luego **Validamos** y ejecutamos nuestro pipeline utilizando **Debug**

<center><img src="https://i.postimg.cc/x85brvKG/adf497.png"></center>

Podemos ver que se han copiado todos los archivos con extensión **.csv** desde la ruta **\host** en nuestro entorno local hacia ADLS a la ruta **raw/ingest**

<center><img src="https://i.postimg.cc/ncdQ01Vp/adf498.png"></center>

#### Crearemos un Pipeline más eficiente realizando una **Carga Incremental (Incremental load)** de archivos basandonos en la última fecha modificada

- **Carga Incremental (Incremental load)**

    * Actividad de CARGAR SOLO NUEVOS REGISTROS desde un origen a un Destino.
    * La carga incremental es mucho más rápida que la carga completa (Full load) y también consume relativamente menos recursos.
    * Requiere menos tiempo.
    

##### **Carga Incremental: Primera Forma**

- **Carga incremental de archivos usando la última fecha modificada**

    * Los archivos se añadirán a la fuente diariamente
    * Usando la última fecha modificada del archivo podemos hacer carga incremental de datos
    * Azure Data Factory seleccionará sólo los archivos modificados de acuerdo a la fecha que nosotros le indiquemos

Podemos ver que el último archivo modificado que fue copiado a ADLS fue **2023-02-16 094712.950119.csv**. Ahora agregamos el archivo **2023-02-23 200147.874793.csv** al directorio **\Host** de nuestro sistema local. Este vendria a ser el último archivo modificado en nuestro sistema local.

<center><img src="https://i.postimg.cc/qRNH8478/adf499.png"></center>

Para efectos del ejemplo, suponer que la fecha actual (la fecha de hoy) es 23/02/2023, por tanto, filtraremos por la última fecha modificada, indicaremos que queremos que filtre SOLO POR EL DIA DE HOY, entonces para **Start time** utilizamos contenido dinámico **@startOfDay(utcnow())**:
```
Start time (UTC): @startOfDay(utcnow()) ---> utcnow(): "2023-02-23T13:34:17.0000000Z"
                                        ---> startOfDay(utcnow()) --> startOfDay('2023-02-23T13:34:17.0000000Z') 
                                                                  --> "2023-02-23T00:00:00.0000000Z"
```
Y para **End time** indicaremos:
```
End time (UTC): @utcnow() ---> utcnow(): "2023-02-23T13:34:17.0000000Z"
```
Utilizando la opción **Filter by last modified** de esta manera solo filtrará archivos que fueron modificados en la fuente de origen en el dia de hoy **2023-02-23**

<center><img src="https://i.postimg.cc/vTXKjKDQ/adf500.png"></center>

De esta manera solo se cargará ESE ARCHIVO ÚNICO y NO TODOS LOS ARCHIVOS NUEVAMENTE

<center><img src="https://i.postimg.cc/W14dQqYj/adf501.png"></center>


##### **Carga Incremental: Segunda Forma**

- **Carga incremental de archivos usando la fecha en el nombre del archivo**

    * También podemos realizar la carga incremental utilizando el formato del nombre del archivo 
    * Extrayendo la fecha del nombre del archivo se puede realizar una carga incremental

<center><img src="https://i.postimg.cc/s2X0Qmn8/adf502.png"></center>

Vamos a empezar creando un nuevo Pipeline que realice esta carga incremental, se llamará **PL_Ingest_Incremental_Load**.
Primero agregaremos una actividad **Get Metadata**

<center><img src="https://i.postimg.cc/0Q2Tdm8M/adf503.png"></center>

Luego agregaremos una actividad **For each** que capturará los **childItems** que devuelve la actividad de **Get Metadata**. Los **childItems** corresponden a los nombres de los archivos que se encuentran en la fuente de origen, es por eso, que la actividad **Get Metadata** utiliza el dataset de origen.
```
@activity('Get Metadata1').output.childItems
```
<center><img src="https://i.postimg.cc/W1stLW8Q/adf504.png"></center>

Vamos a trabajar en la actividad **For each**

<center><img src="https://i.postimg.cc/1zkrH10k/adf505.png"></center>

Tenemos la siguiente **Expression**:
```
equals(): Devuelve "true" si dos valores son iguales
item(): esta referenciando a "Items" de Settings
item().name: dado que "Items" captura los "childItems" que devuelve la actividad "Get Metadata", 'name' hace referencia al nombre del archivo

@equals(
    substring(item().name,0,10),
    formatDateTime(utcnow(),yyyy-MM-dd)
    )

substring(item().name,0,10) --> substring('2023-02-23 200147.874793.csv',0,10) --> "2023-02-23"
formatDateTime(utcnow(),yyyy-MM-dd) --> formatDateTime('2023-02-23T13:34:17.0000000Z', yyyy-MM-dd) --> "2023-02-23"
```

<center><img src="https://i.postimg.cc/W306YYXh/adf506.png"></center>

Vamos a trabajar en la opción de **True**

<center><img src="https://i.postimg.cc/q72TJnLW/adf507.png"></center>

Previamente, debemos crear un nuevo dataset de origen llamado **DS_Onprem_File_param** que utilizará parámetros. También hará referencia a **File system** y al tipo de archivo **Delimited Text**. Utilizará el Linked Service de origen y la misma ruta hacia **\Host**. Debemos marcar la casilla de **First Row as Header**

<center><img src="https://i.postimg.cc/7hzH2cMY/adf508.png"></center>

Creamos un parámetro a nivel de dataset de origen

<center><img src="https://i.postimg.cc/qMM00zWv/adf509.png"></center>
<center><img src="https://i.postimg.cc/Y9zKBH3F/adf510.png"></center>

Debemos crear un nuevo dataset de destino llamado **DS_ADLS_ingest_param** que utilizará parámetros. También hará referencia a **Azure Data Lake Storage Gen2** y al tipo de archivo **Delimited Text**. Utilizará el Linked Service de destino y la misma ruta hacia **raw/ingest**

<center><img src="https://i.postimg.cc/CMCDXQVV/adf511.png"></center>

Creamos un parámetro a nivel de dataset de destino

<center><img src="https://i.postimg.cc/d3Q0zn8q/adf512.png"></center>
<center><img src="https://i.postimg.cc/xTCC5xf5/adf513.png"></center>

El parámetro del dataset de origen **FileName** tomará el valor **@{item().name}**

<center><img src="https://i.postimg.cc/c4yKxM0B/adf514.png"></center>

El parámetro del dataset de destino **FileName** tomará el valor **@{item().name}**

<center><img src="https://i.postimg.cc/RFJF9qYY/adf515.png"></center>
<center><img src="https://i.postimg.cc/DfsZMJLJ/adf516.png"></center>

Antes de **Validar** y ejecutar con **Debug**, eliminamos el archivo **2023-02-16 094712.950119.csv** de nuestro ADLS, cargado anteriormente. Al ejecutarse, irá SECUENCIALMENTE (como indicamos en la actividad **For each**) archivo por archivo e irá verificando la **Expresión**. Vemos que el ÚLTIMO ARCHIVO ejecutó el **Copy data** dado que es el único archivo que cumplío la Expresión de la actividad **If Condition**

<center><img src="https://i.postimg.cc/s2vf9v32/adf517.png"></center>
<center><img src="https://i.postimg.cc/mZQLwxW7/adf518.png"></center>